# Blackfynn Notebook

[Click here to go on the Blackfynn webpage](https://www.blackfynn.com/)

## 1. Log in to your account

In [7]:
from blackfynn import Blackfynn # Load the Blackfynn python client library
import ipywidgets as widgets # load library for interactive widgets (drop-down lists, button, etc.)
from ipywidgets import HBox, Label, Layout
from IPython.core.display import display, HTML
from IPython.display import clear_output, Markdown

### Define some useful functions

# Print with style (bold etc.)
def printmd(string): 
    display(Markdown(string))

# Button
def mybutton(mydescription):
    mybuttonis = widgets.Button(
    description = mydescription,
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''    
    )
    return mybuttonis

# Dropdown list
def mydropdownlist(listoptions):
    mydropdownlistis = widgets.Dropdown(
    options = listoptions,
    value = listoptions[0],
    disabled = False    
    )
    return mydropdownlistis

# Textbox
def mytextbox(placeholder):
    mytextboxis = widgets.Text(
        #value='Type dataset name',
        placeholder=placeholder,
        disabled=False,
        layout=Layout(width='60%')
    )
    return mytextboxis

### Profile setup instructions
printmd('**Setup your Balckfynn profile on your computer following the instructions available here:**') 
#display(HTML('<h1>Hello, world!</h1>'))('http://help.blackfynn.com/developers/configuration/creating-an-api-key-for-the-blackfynn-clients')
display(HTML("<a href='http://help.blackfynn.com/developers/configuration/creating-an-api-key-for-the-blackfynn-clients'>http://help.blackfynn.com/developers/configuration/creating-an-api-key-for-the-blackfynn-clients</a>"))

### Check profile setup
print()
print()
checkprofile_button = mybutton('Check profile setup')
check_box = HBox([Label('Click here to check your profile:'), checkprofile_button])
display(check_box)

output = widgets.Output()
@output.capture()
def checkprofile_button_clicked(b):
    clear_output()
    print('Checking...')
    clear_output(wait=True)
    !bf status
    bf = Blackfynn() # Connect to your profile
checkprofile_button.on_click(checkprofile_button_clicked)
display(output)


**Setup your Balckfynn profile on your computer following the instructions available here:**

Output()

## 2. Look at your existing dataset and files on Blackfynn

In [19]:
import pandas as pd # for conveniently displaying the files in a dataset

bf = Blackfynn() 

seedataset_button = mybutton('Show existing dataset')
check_box = HBox([Label('Click here to see your dataset:'), seedataset_button])
display(check_box)

showdataset_button = mybutton("Dataset structure")

output = widgets.Output()
@output.capture()
def seedataset_button_clicked(b):
    clear_output()
    dataset_list = ['See datasets']
    for ds in bf.datasets():
        dataset_list.append(ds.name) 
    showdataset(dataset_list)
seedataset_button.on_click(seedataset_button_clicked)
display(output)

def showdataset(dataset_option):
    dataset_dropdown = mydropdownlist(dataset_option)
    dataset_selection_box = HBox([Label('Select dataset:'), dataset_dropdown, showdataset_button])
    display(dataset_selection_box)
    output = widgets.Output()
    @output.capture()
    def showdataset_button_clicked(b):
        if (dataset_dropdown.value == 'See datasets'):
            clear_output()
            printmd('**Please select a dataset**')
        else:
            clear_output()
            mydataset = bf.get_dataset(dataset_dropdown.value)
            print_file_structure(mydataset,0)
    showdataset_button.on_click(showdataset_button_clicked)
    display(output)  

def print_file_structure(dataset, indent):
    if (indent==0):
        print('{0}{1}'.format("Dataset = ", dataset.name))
    else:
        print('{0} * {1} ({2})'.format(' '*indent, dataset.name, 'Folder'))          
    for item in dataset.items:
        try: 
            item._properties["Blackfynn"]["subtype"].value
        except:
            print_file_structure(item, indent=indent+5)
        else:
            print('{0} . {1} ({2})'.format(' '*(indent+5), item.name, item._properties["Blackfynn"]["subtype"].value))

Output()

## 3. Create dataset

In [9]:
### Enter data set name
newdatasetname_textbox = mytextbox('Type new dataset name')
datasetname_box = HBox([Label('Enter name:'), newdatasetname_textbox])
display(datasetname_box)

### Create dataset
createdataset_button = mybutton('Create dataset')
createdataset_box = HBox([createdataset_button])
display(createdataset_box)


import traceback
output = widgets.Output()
@output.capture()
def createdataset_button_clicked(b):
    clear_output()
    if (newdatasetname_textbox.value ==''):
        printmd('**Please enter a name**')
    else:
        try:
            ds = bf.create_dataset(newdatasetname_textbox.value)
        except:
            exceptiondata = traceback.format_exc().splitlines()
            exceptionarray = [exceptiondata[-1]] 
            print(exceptionarray)
        else:
            printmd('**Dataset created. New dataset list:**')
            count = 0
            for ds in bf.datasets():
                count += 1
                print("Dataset # {0}: ".format(count), ds.name)
    
createdataset_button.on_click(createdataset_button_clicked)
display(output)

Output()

## 4. Upload a file to an existing dataset

In [18]:
# Enter data set name
datasetname_textbox = mytextbox('Type dataset name')
datasetname_box = HBox([Label('Dataset name:'), datasetname_textbox])
display(datasetname_box)

#Specify file location
datafilename_textbox = mytextbox(r'e.g. C:\User\Desktop\DOPEv1.1\examples\blackfynn\Hello.docx')
datafilename_box = HBox([Label('File location:'), datafilename_textbox])
display(datafilename_box)

#Upload
uploaddatafile_button = mybutton('Upload file')
uploaddatafile_box = HBox([uploaddatafile_button])
display(uploaddatafile_box)

import traceback
output = widgets.Output()
@output.capture()
def uploaddatafile_button_clicked(b):
    clear_output()
    print('Uploading...')
    try:
        ds = bf.get_dataset(datasetname_textbox.value)
    except:
        clear_output()
        printmd('**Dataset not found**')
        exceptiondata = traceback.format_exc().splitlines()
        exceptionarray = [exceptiondata[-1]] 
        print(exceptionarray)
    else:
        try:
            path = datafilename_textbox.value
            ds.upload(path)  
        except:
            clear_output()
            printmd('**File could not be located**')
        else:
            clear_output()
            printmd('**File uploaded**')
    
uploaddatafile_button.on_click(uploaddatafile_button_clicked)
display(output)

# example path = r'C:\Users\Calmi2\Desktop\My data\Hello.docx'

Output()

## 5. Upload a folder of files (folder structure conserved)

In [26]:
import os
from os import listdir, walk
from os.path import isfile, join

# Enter data set name
datasetname_textbox = mytextbox('Type dataset name')
datasetname_box = HBox([Label('Dataset name:'), datasetname_textbox])
display(datasetname_box)

#Specify file location
datafilename_textbox = mytextbox('e.g. C:/User/Desktop/My data')
datafilename_box = HBox([Label('Folder location:'), datafilename_textbox])
display(datafilename_box)

#Upload
uploaddatafile_button = mybutton('Upload folder')
uploaddatafile_box = HBox([uploaddatafile_button])
display(uploaddatafile_box)

import traceback
output = widgets.Output()
@output.capture()
def uploaddatafile_button_clicked(b):
    clear_output()
    print('Uploading...')
    try:
        ds = bf.get_dataset(datasetname_textbox.value)
    except:
        clear_output()
        printmd('**Dataset not found**')
        #exceptiondata = traceback.format_exc().splitlines()
        #exceptionarray = [exceptiondata[-1]] 
        #print(exceptionarray)
    else:
        #path = datafilename_textbox.value
        #upload_structured_file(ds, path, ds.name, 0)
        try:
            path = datafilename_textbox.value
            upload_structured_file(ds, path, ds.name, 0)
        except:
            #clear_output()
            printmd('**Folder could not be located**')
        else:
            printmd('**Folder and associated files uploaded**')

def upload_structured_file(myds, mypath, myfolder, foldercount):
    mypath = os.path.join(mypath)
    for f in listdir(mypath):
        if isfile(join(mypath, f)):  
            print('Uploading', f, 'in', myfolder)
            filepath = os.path.join(mypath, f)
            myds.upload(filepath)
            print(f, 'uploaded')
        else:
            print('Creating folder', f)
            mybffolder = myds.create_collection(f)
            print(f, 'folder created')
            myfolderpath = os.path.join(mypath, f)
            foldercount += 1
            upload_structured_file(mybffolder, myfolderpath, f, foldercount)
uploaddatafile_button.on_click(uploaddatafile_button_clicked)
display(output)

# C:\User\Desktop\DOPEv1.1\examples\blackfynn\example_data

Output()

## 6. Create a calendar reminder

Create a calendar event file (.ics) that could be imported in any calendar (Google, Outlook, etc.)

In [13]:
import smtplib
from datetime import datetime
from threading import Timer
import os

### Dataset name
datasetreminder_textbox = mytextbox('Remember to...')

datesetreminder_box = HBox([Label('Enter reminder message:'), datasetreminder_textbox])
display(datesetreminder_box)

### Specify date and time

# Specify date
reminder_datepicker = widgets.DatePicker(
    #description='Pick a Date',
    disabled=False
)

# Specify time
reminder_timepicker = widgets.DatePicker(
    #description='Pick a Date',
    disabled=False
)

year_textbox = widgets.BoundedIntText(
    #value=7,
    min=2018,
    max=3000,
    step=1,
    placeholder = 2018,
    #description='Text:',
    disabled=False,
    layout=Layout(width='10%')
)

month_textbox = widgets.BoundedIntText(
    #value=7,
    min=1,
    max=12,
    step=1,
    placeholder = 1,
    #description='Text:',
    disabled=False,
    layout=Layout(width='10%')
)

day_textbox = widgets.BoundedIntText(
    #value=7,
    min=1,
    max=31,
    step=1,
    placeholder = 1,
    #description='Text:',
    disabled=False,
    layout=Layout(width='10%')
)

hour_textbox = widgets.BoundedIntText(
    #value=7,
    min=0,
    max=24,
    step=1,
    placeholder = 13,
    #description='Text:',
    disabled=False,
    layout=Layout(width='10%')
)

min_textbox = widgets.BoundedIntText(
    #value=7,
    min=0,
    max=60,
    step=1,
    placeholder = 30,
    #description='Text:',
    disabled=False,
    layout=Layout(width='10%')
)

# Box
datetime_box = HBox([Label('Enter reminder date and time:'), year_textbox, month_textbox, day_textbox, hour_textbox, min_textbox])
display(datetime_box)


### Button test then validate reminder setup

testemail_button = widgets.Button(
    description='Create calendar event',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''    
)

reminder_box = HBox([testemail_button])
display(reminder_box)

from ics import Calendar, Event
output = widgets.Output()
@output.capture()
def testemail_button_clicked(b):
    clear_output()
    print('Creating calendar event')

    datasetname = datasetreminder_textbox.value
    
    cwd = os.getcwd()
    mypath = os.path.join(cwd + '\\' + 'calendar_event' + '.ics' )
    #print(mypath)
    c = Calendar()
    e = Event()
    reminderdate = datetime(year = year_textbox.value, month = month_textbox.value, day = day_textbox.value, hour = hour_textbox.value+5, minute = min_textbox.value)
    e.name = datasetreminder_textbox.value
    e.begin = reminderdate
    c.events.add(e)
    c.events
    #print(c.events)
    with open(mypath, 'w') as f:
        f.writelines(c)
    print('Event created in file calendar_event.ics')

    
testemail_button.on_click(testemail_button_clicked)
display(output)

Output()